# Tarea 3: clasificar fundamentos segun su modo

## Consideramos los modelos precomputados

<p>Carga los modelos creados previamente (OJO, cargar los modelos de vectores preentrendos puede tardar mucho y usar mucha memoria porque pesa 1GB cada uno)</p>

In [7]:
# we assume models are already created
# these are the global variables used when computing all the metrics
# best models in with no pretrained vectors in _model_<tema>_best_at_<k>.bin
# best models in with pretrained vectors in _model_<tema>_best_at_<k>.bin

import fasttext

_dataDir = "../../data/tarea3/"
_models_task3 = "./"
ftlabel = "__label__"

# prefix and suffixes for the models to load
model_no_ptvec = "_model_"
model_ptvec = "_model_ptvec_"
model_no_ptvec_f1 = "_model_f1_"
model_no_ptvec_f1_wn2 = "_model_f1_wn2_"
model_no_ptvec_f1_wn3 = "_model_f1_wn3_"
model_ptvec_f1 = "_model_f1_ptvec_"
model_ptvec_f1_wn2 = "_model_f1_ptvec_wn2_"

# what models to consider
# m_prefixes = [model_no_ptvec]
model_names = [model_no_ptvec, model_ptvec, model_no_ptvec_f1, model_ptvec_f1, 
               model_no_ptvec_f1_wn2, model_no_ptvec_f1_wn3, model_ptvec_f1_wn2]
model_labels = ['npt','ptv','nf1','pf1','nf2','nf3','pf2']

<p>Ahora carga los datos para ajustar los modelos. Cargamos solo test set y dev set</p>

In [10]:
# load data to predict

import string

def read_text_file_for_ft_input(filename):
    with open(filename) as f:
        out = []
        for line in f:
            # keep tab to separate original concepts from justifications
            strdata = "".join([c for c in line[:-1] if c not in string.punctuation or c == '\t']).lower()
            if strdata == '':
                strdata = ' '
            out.append(strdata)
    return out

def read_numbers_file_for_ft_input(filename):
    with open(filename) as f:
        out = []
        for line in f:
            out.append(int(line))
    return out

train_x = {}
train_y = {}

test_x = {}
test_y = {}

dev_x = {}
dev_y = {}


train_x = read_text_file_for_ft_input(
        _dataDir + "x_train_modo.txt")
train_y = read_numbers_file_for_ft_input(
        _dataDir + "y_train_modo.txt")
test_x = read_text_file_for_ft_input(
        _dataDir + "x_test_modo.txt")
test_y = read_numbers_file_for_ft_input(
        _dataDir + "y_test_modo.txt")
dev_x = read_text_file_for_ft_input(
        _dataDir + "x_dev_modo.txt")
dev_y = read_numbers_file_for_ft_input(
        _dataDir + "y_dev_modo.txt")

modos = []
# load categories first
modosFile = _dataDir + "modos.txt"
with open(modosFile) as f:
    for line in f:
        modos.append(line[:-1])

<p>Computa las predicciones con cada uno de los modelos</p>

In [11]:
size = 3

predictions_train_with_labels = {}
predictions_dev_with_labels = {}
predictions_with_labels = {}

for model_name in model_names:
    model = fasttext.load_model(_models_task3 + model_name + "best.bin")
    predictions_with_labels[model_name] = model.predict(
            test_x,k = size)
    predictions_dev_with_labels[model_name] = model.predict(
            dev_x,k = size)
    predictions_train_with_labels[model_name] = model.predict(
            train_x,k = size)

    
predictions = {}
predictions_dev = {}
predictions_train = {}

# cambia las predicciones del tipo <ftlabel><C> a simplemente <C>
for model_name in model_names:
    predictions_dev[model_name] = []
    predictions[model_name] = []
    predictions_train[model_name] = []
    for pred_list_labels in predictions_with_labels[model_name]:
        pred_list = []
        for label in pred_list_labels:
            pred_list.append(int(label[len(ftlabel):]))
        predictions[model_name].append(pred_list)
    predictions_dev[model_name] = []
    for pred_list_labels in predictions_dev_with_labels[model_name]:
        pred_list = []
        for label in pred_list_labels:
            pred_list.append(int(label[len(ftlabel):]))
        predictions_dev[model_name].append(pred_list)
    for pred_list_labels in predictions_train_with_labels[model_name]:
        pred_list = []
        for label in pred_list_labels:
            pred_list.append(int(label[len(ftlabel):]))
        predictions_train[model_name].append(pred_list)

            
# utility function to select the first prediction from a list of predictions and generate a 1D list of single predictions
def first_prediction(lists_of_predictions):
    out = []
    for predictions in lists_of_predictions:
        out.append(predictions[0])
    return out      

## Genera reportes

In [12]:
from sklearn import metrics


print("\t\trecavg\tprecavg\tf1avg\tf1macro")
for model, model_name in zip(model_names,model_labels):
    for pred,gold,set_name in zip(
        [predictions_train[model],predictions_dev[model],predictions[model]],
        [train_y,dev_y,test_y],
        ['train','devs','test']):
        prediction = first_prediction(pred)
        str_data = set_name + "-" + model_name
#        acc = round(100*metrics.accuracy_score(gold,prediction),2)
#        str_data += "\t"+ str(acc)
        rec = round(100*metrics.recall_score(gold,prediction,average='weighted'),2)
        str_data += "\t"+ str(rec)
        prec = round(100*metrics.precision_score(gold,prediction,average='weighted'),2)
        str_data += "\t"+str(prec)
        f1a = round(100*metrics.f1_score(gold,prediction,average='weighted'),2)
        str_data += "\t"+str(f1a)
        f1m = round(100*metrics.f1_score(gold,prediction,average='macro'),2)
        str_data += "\t"+str(f1m)

        print(str_data)
print()

            

		recavg	precavg	f1avg	f1macro
train-npt	82.07	80.64	80.73	65.59
devs-npt	80.65	78.93	79.13	62.72
test-npt	80.93	79.23	79.41	62.85
train-ptv	82.04	80.57	80.66	65.03
devs-ptv	80.87	79.14	79.33	62.6
test-ptv	81.08	79.42	79.61	63.01
train-nf1	82.83	81.59	81.71	67.73
devs-nf1	80.48	78.81	79.14	63.05
test-nf1	80.98	79.38	79.68	63.28
train-pf1	82.11	80.67	80.83	65.45
devs-pf1	80.82	79.14	79.38	62.99
test-pf1	81.08	79.45	79.7	63.29
train-nf2	91.02	90.72	90.74	82.64
devs-nf2	80.64	79.35	79.77	64.08
test-nf2	81.08	79.84	80.22	64.79
train-nf3	98.87	98.87	98.87	97.84
devs-nf3	80.17	78.79	79.23	63.09
test-nf3	80.04	78.64	79.1	62.69
train-pf2	89.54	89.14	89.15	80.1
devs-pf2	81.21	79.88	80.26	64.69
test-pf2	81.57	80.31	80.66	65.37

